In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

# Astronomy
import astropy
from astropy.units import au, minute

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_elements, load_ast_vectors
from asteroid_spline import get_df_shape, spline_ast_data, make_spline_df
from asteroid_direction import calc_dir_ast2obs
from astro_utils import deg2dist, dist2sec
from orbital_element import elt2pos
from planets_interp import get_sun_pos
from db_utils import sp2df, df2db

In [3]:
from detection_near_ast import get_data_detections, get_data_ast_dir, asteroid_batch_prelim, asteroid_batch_vec, asteroid_batch_elt, light_time_adj
from detection_near_ast import cols_u_obs, cols_q_obs, cols_u_ast, cols_v_ast, cols_q_ast, mjd0, mjd1, c

In [4]:
# Inputs
d0 = 1
d1 = 1000
n0 = 1
n1 = 10
deg_max = 180
arcmin_max = deg_max * 180

In [5]:
# Get detections
det = get_data_detections(d0=d0, d1=d1)
det

,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z
DetectionID,,,,,,,,
1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032
2,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032
3,3,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032
4,4,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032
5,5,58270.167662,-0.944884,-0.197557,0.261085,-0.342267,-0.947311,-0.000032
...,...,...,...,...,...,...,...,...
995,995,58270.169965,-0.956204,0.008802,0.292568,-0.342231,-0.947325,-0.000032
996,996,58270.169965,-0.955604,0.003942,0.294628,-0.342231,-0.947325,-0.000032
997,997,58270.169965,-0.957723,0.000489,0.287693,-0.342231,-0.947325,-0.000032


In [6]:
# Get asteroid directions
ast_in = get_data_ast_dir(n0=n0, n1=n1)

# Number of distinct asteroids in the input data
ast_unq = np.unique(ast_in.AsteroidID)
N_ast = ast_unq.size

# Build data frame with index = AsteroidID, payload n (asteroid number in this batch)
dfa_tbl = {
    'AsteroidID': ast_unq,
    'n': np.arange(N_ast, dtype=np.int)
}
dfa = pd.DataFrame(dfa_tbl)    
dfa.set_index(keys='AsteroidID', drop=False, inplace=True)

In [7]:
dfa

,AsteroidID,n
AsteroidID,,
1,1,0
2,2,1
3,3,2
4,4,3
5,5,4
6,6,5
7,7,6
8,8,7
9,9,8


In [6]:
# First pass at detections near asteroids
dna_pre = asteroid_batch_prelim(det=det, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [7]:
dna_pre

,AsteroidID,DetectionID,n,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,LightTime,uAst_x,uAst_y,uAst_z,s
0,1,1,0,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.167662,22.718352,-0.750616,0.638384,0.170414,0.870543
1,1,2,0,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.167662,22.718352,-0.750616,0.638384,0.170414,0.872219
2,1,3,0,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.167662,22.718352,-0.750616,0.638384,0.170414,0.866530
3,1,4,0,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.167662,22.718352,-0.750616,0.638384,0.170414,0.863526
4,1,5,0,58270.167662,-0.944884,-0.197557,0.261085,-0.342267,-0.947311,-0.000032,58270.167662,22.718352,-0.750616,0.638384,0.170414,0.862994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8986,9,995,8,58270.169965,-0.956204,0.008802,0.292568,-0.342231,-0.947325,-0.000032,58270.169965,14.108814,-0.017399,-0.999234,-0.035061,1.415921
8987,9,996,8,58270.169965,-0.955604,0.003942,0.294628,-0.342231,-0.947325,-0.000032,58270.169965,14.108814,-0.017399,-0.999234,-0.035061,1.412546
8988,9,997,8,58270.169965,-0.957723,0.000489,0.287693,-0.342231,-0.947325,-0.000032,58270.169965,14.108814,-0.017399,-0.999234,-0.035061,1.409903
8989,9,998,8,58270.169965,-0.957586,-0.000229,0.288147,-0.342231,-0.947325,-0.000032,58270.169965,14.108814,-0.017399,-0.999234,-0.035061,1.409407


In [12]:
dna_pre['n'] = dfa.n[dna_pre.AsteroidID].values

In [11]:
np.unique(dna_pre.AsteroidID)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
ast_unq = np.unique(dna_pre.AsteroidID)
num_ast = ast_unq.shape[0]
dfa_tbl = {
    'AsteroidID': ast_unq,
    'n': np.arange(num_ast, dtype=np.int)
}
dfa = pd.DataFrame(dfa_tbl)
dfa.set_index(keys='AsteroidID', drop=False, inplace=True)
dfa
dna_pre['n'] = dfa[dfa_pre.AsteroidID]

NameError: name 'dfa_pre' is not defined

In [8]:
# Sharpen estimate using splined asteroid vectors
dna_vec = dna_pre.copy()
asteroid_batch_vec(dna=dna_vec, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [9]:
dna_vec

,AsteroidID,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,LightTime,uAst_x,uAst_y,uAst_z,s,qAst_x,qAst_y,qAst_z
0,1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.870914,-2.389672,0.795615,0.465448
1,1,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.872591,-2.389672,0.795615,0.465448
2,1,3,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.866904,-2.389672,0.795615,0.465448
3,1,4,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.863901,-2.389672,0.795615,0.465448
4,1,5,58270.167662,-0.944884,-0.197557,0.261085,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.863370,-2.389672,0.795615,0.465448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5989,9,995,58270.169965,-0.956204,0.008802,0.292568,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.417024,-0.368985,-2.643489,-0.059527
5990,9,996,58270.169965,-0.955604,0.003942,0.294628,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.413651,-0.368985,-2.643489,-0.059527
5991,9,997,58270.169965,-0.957723,0.000489,0.287693,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.411012,-0.368985,-2.643489,-0.059527
5992,9,998,58270.169965,-0.957586,-0.000229,0.288147,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.410516,-0.368985,-2.643489,-0.059527


In [10]:
# Report light time error
_  = light_time_adj(df=dna_vec, verbose=True)

Mean light time error: 2.46e-09 minutes


In [11]:
# Sharpen estimate using splined asteroid elements
dna_elt = dna_pre.copy()
asteroid_batch_elt(dna=dna_elt, n0=n0, n1=n1, arcmin_max=arcmin_max)

In [12]:
dna_elt

,AsteroidID,DetectionID,tObs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z,tAst,LightTime,uAst_x,uAst_y,uAst_z,s,qAst_x,qAst_y,qAst_z
0,1,1,58270.167662,-0.940203,-0.205183,0.271880,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.870914,-2.389672,0.795615,0.465448
1,1,2,58270.167662,-0.940593,-0.207164,0.269011,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.872591,-2.389672,0.795615,0.465448
2,1,3,58270.167662,-0.942678,-0.201138,0.266274,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.866904,-2.389672,0.795615,0.465448
3,1,4,58270.167662,-0.944177,-0.198029,0.263275,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.863901,-2.389672,0.795615,0.465448
4,1,5,58270.167662,-0.944884,-0.197557,0.261085,-0.342267,-0.947311,-0.000032,58270.151902,22.694705,-0.750296,0.638716,0.170581,0.863370,-2.389672,0.795615,0.465448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5989,9,995,58270.169965,-0.956204,0.008802,0.292568,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.417024,-0.368985,-2.643489,-0.059527
5990,9,996,58270.169965,-0.955604,0.003942,0.294628,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.413651,-0.368985,-2.643489,-0.059527
5991,9,997,58270.169965,-0.957723,0.000489,0.287693,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.411012,-0.368985,-2.643489,-0.059527
5992,9,998,58270.169965,-0.957586,-0.000229,0.288147,-0.342231,-0.947325,-0.000032,58270.160162,14.116994,-0.015762,-0.999261,-0.035050,1.410516,-0.368985,-2.643489,-0.059527


In [25]:
np.unique(dna_pre.AsteroidID)

array([1, 2, 4, 7, 8, 9])

In [24]:
deg2dist(1.0)

0.01745307099674787